In [ ]:
import kplr
import numpy as np
# import matplotlib.pyplot as plt

from functools import partial
from lmfit     import Parameters, Minimizer, report_errors, minimize

from matplotlib import rcParams
from time      import time

from sklearn.externals import joblib
from scipy import special
from pandas import DataFrame
from exoparams import PlanetParams
import batman

# plt.ion()
# %matplotlib inline

In [ ]:
from bokeh.io       import output_notebook, show
from bokeh.plotting import figure
from bokeh.models   import Span
from bokeh.layouts  import gridplot

output_notebook()

In [ ]:
color_cycle = rcParams['axes.prop_cycle'].by_key()['color']

**LMFIT BATMAN Model**

In [ ]:
def batman_wrapper_lmfit(period, tCenter, inc, aprs, rprs, edepth, ecc, omega, u1, u2, 
                         intcpt, slope, crvtur, times, ldtype='quadratic', transitType='primary'):
    
    if intcpt == 1.0 and slope == 0.0 and crvtur == 0.0:
        OoT_crvtur = 1.0 # OoT == Out of Transit
    else:
        OoT_crvtur = intcpt + slope*(times-times.mean()) + crvtur*(times-times.mean())**2
    
    bm_params           = batman.TransitParams() # object to store transit parameters
    
    bm_params.per       = period   # orbital period
    bm_params.t0        = tCenter  # time of inferior conjunction
    bm_params.inc       = inc      # inclunaition in degrees
    bm_params.a         = aprs     # semi-major axis (in units of stellar radii)
    bm_params.rp        = rprs     # planet radius (in units of stellar radii)
    bm_params.fp        = edepth   # planet radius (in units of stellar radii)
    bm_params.ecc       = ecc      # eccentricity
    bm_params.w         = omega    # longitude of periastron (in degrees)
    bm_params.limb_dark = ldtype   # limb darkening model # NEED TO FIX THIS
    bm_params.u         = [u1, u2] # limb darkening coefficients # NEED TO FIX THIS
    
    m_eclipse = batman.TransitModel(bm_params, times, transittype=transitType)# initializes model
    
    return m_eclipse.light_curve(bm_params)*OoT_crvtur

In [ ]:
from exoplanet_bokeh_plots import batman_wrapper_lmfit

''' Example Usage for `batman_wrapper_lmfit`

keplaunch = 2454833.0 # Kepler time stamps are relative to the Julian Date (JD) of launch 

# From Hubert et al. 2017
kep3period = 4.88782433
kep3t0     = 2454957.812464 - keplaunch
kep3aoR    = 14.64
kep3RpRs   = 0.05856
kep3FpFs   = 500/ppm
kep3inc    = 88.99
kep3ecc    = 0.26493
kep3omeg   = -162.149
kep3u1     = 0.646 # linear_limb_darkening
kep3u2     = 0.048 # quad_limb_darkening

intcpt     = 1.0   # out of transit intercept
slope      = 0.0   # out of transit slope
crvtur     = 0.0   # out of transit curvature

n_data     = 1000
times_array= np.linspace(kep3t0 - 0.1*kep3period, kep3t0 + 0.1*kep3period, n_data)

usage = batman_wrapper_lmfit(kep3period, kep3t0, inclination, kep3aoR, kep3RpRs, kep3FpFs, 
                                kep3inc, kep3omeg, kep3u1, kep3u2, intcpt, slope, crvtur, 
                                 times_array, ldtype='quadratic', transitType='primary')
'''

In [ ]:
from exoplanet_bokeh_plots import transit_line_model

''' Example Usage for `batman_wrapper_lmfit`

keplaunch = 2454833.0 # Kepler time stamps are relative to the Julian Date (JD) of launch 

# From Hubert et al. 2017
kep3period = 4.88782433
kep3t0     = 2454957.812464 - keplaunch
kep3aoR    = 14.64
kep3RpRs   = 0.05856
kep3FpFs   = 500/ppm
kep3inc    = 88.99
kep3ecc    = 0.26493
kep3omeg   = -162.149
kep3u1     = 0.646 # linear_limb_darkening
kep3u2     = 0.048 # quad_limb_darkening

intcpt     = 1.0   # out of transit intercept
slope      = 0.0   # out of transit slope
crvtur     = 0.0   # out of transit curvature

model_params = Parameters()

model_params.add_many(
    ('period'   , kep3period, False),
    ('tCenter'  , kep3t0    , True , kep3t0 - 0.1, kep3t0 + 0.1),
    ('inc'      , kep3inc   , False, 0.0, 90. ),
    ('aprs'     , kep3aoR   , False, 0.0, 100.),
    ('tdepth'   , kep3RpRs  , True , 0.0, 1.0 ),
    ('edepth'   , kep3FpFs  , True , 0.0, 1.0 ),
    ('ecc'      , kep3ecc   , False, 0.0, 1.0 ),
    ('omega'    , kep3omeg  , False, 0.0, 1.0 ),
    ('u1'       , kep3u1    , True , 0.0, 1.0 ),
    ('u2'       , kep3u2    , True , 0.0, 1.0 ),
    ('intcpt'   , 1.0       , True ),
    ('slope'    , 0.0       , True ),
    ('crvtur', 0.0          , True ))

n_data     = 1000
times_array= np.linspace(kep3t0 - 0.1*kep3period, kep3t0 + 0.1*kep3period, n_data)

usage = transit_line_model(model_params, times_array)
'''

In [ ]:
def transit_line_model(model_params, times):
    intcpt  = model_params['intcpt'].value if 'intcpt' in model_params.keys() else 1.0
    slope   = model_params['slope'].value  if 'slope'  in model_params.keys() else 0.0
    crvtur  = model_params['crvtur'].value if 'crvtur' in model_params.keys() else 0.0
    
    # Transit Parameters
    period  = model_params['period'].value
    tCenter = model_params['tCenter'].value
    inc     = model_params['inc'].value
    aprs    = model_params['aprs'].value
    edepth  = model_params['edepth'].value
    tdepth  = model_params['tdepth'].value
    ecc     = model_params['ecc'].value
    omega   = model_params['omega'].value
    u1      = model_params['u1'].value
    u2      = model_params['u2'].value
    
    # delta_phase = deltaphase_eclipse(ecc, omega) if ecc is not 0.0 else 0.5
    # t_secondary = tCenter + period*delta_phase
    
    rprs  = np.sqrt(tdepth)
    
    transit_model = batman_wrapper_lmfit(period, tCenter, inc, aprs, rprs, edepth, ecc, omega, u1, u2, 
                         intcpt, slope, crvtur, times, ldtype='quadratic', transitType='primary')
    
    line_model    = intcpt + slope*(times-times.mean()) + crvtur*(times-times.mean())**2.
    
    return transit_model * line_model

In [ ]:
def residuals_func(model_params, times, flux, fluxerr):
    model = transit_line_model(model_params, times)
    return (model - flux) / fluxerr

**Bokeh Plots**

In [ ]:
def bokeh_errorbars(xs, ys, yerrs, xerrs=None, color='#1f77b4', size=5, alpha=1, fig=None, show_now = False):
    
    if xerrs is None:
        xerrs = np.zeros(xs.size)
    
    if fig is None:
        fig = figure()
    
    fig.circle(xs, ys, color=color, size=size, alpha=alpha)
    
    # create the coordinates for the errorbars
    err_xs = []
    err_ys = []
    
    for x, y, yerr, xerr in zip(xs, ys, yerrs, xerrs):
        err_xs.append((x - xerr, x + xerr))
        err_ys.append((y - yerr, y + yerr))
    
    # plot them
    fig.multi_line(err_xs, err_ys, color=color, alpha=alpha)
    
    if show_now: show(fig)
    
    return fig

In [ ]:
def bokeh_hist(data, bins=100, range=None, color='#1f77b4', density=True, alpha=1.0, fig = None, show_now = False):
    if fig is None:
        fig = figure()
    
    data_sorted = np.sort(data)
    hist, edges = np.histogram(data_sorted, density=density, bins=bins, range=range)
    fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], 
             fill_color=color, line_color=color, alpha=alpha)
    
    if show_now: show(fig)
    
    return fig

In [ ]:
def bokeh_corner_plot(dataset, TOOLS=None, hist_color='orange', kde_color="violet"):
    # if dataset.shape[0] > dataset.shape[1]:
    #     raise Exception('Shape must be dimensions x samples -- i.e. (9,1000), not (1000,9)')
    
    if isinstance(dataset, np.ndarray):
        dataset = DataFrame(dataset)
    
    if TOOLS is None:
        TOOLS = "box_select,lasso_select,pan,wheel_zoom,box_zoom,reset,help"
    
    scatter_plots = []
    y_max = len(dataset.columns) - 1
    for i, y_col in enumerate(dataset):
        for j, x_col in enumerate(dataset):
            df = DataFrame({x_col: dataset[x_col].tolist(), y_col: dataset[y_col].tolist()})
            fig = figure(tools=TOOLS, toolbar_location="below", toolbar_sticky=False)
            if i >= j:
                if i != j:
                    fig.scatter(x=x_col, y=y_col, source=df)
                else:
                    x_now       = np.sort(dataset[x_col].values)
                    mu  , sigma = np.mean(x_now), np.std(x_now)
                    hist, edges = np.histogram(x_now, density=True, bins=len(x_now)//100)
                    pdf         = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-0.5*(x_now-mu)**2 / sigma**2)
                    cdf         = 0.5*(1+special.erf((x_now-mu)/np.sqrt(2*sigma**2)))
                    
                    fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color=hist_color, line_color=hist_color, alpha=1.0)
                    fig.line(x_now, pdf, line_color=kde_color, line_width=8, alpha=0.7)#, legend="PDF")
                    #fig.line(x_now, cdf, line_color="black"  , line_width=2, alpha=0.5, legend="CDF")
                if j > 0:
                    fig.yaxis.axis_label = ""
                    fig.yaxis.visible = False
                if i < y_max:
                    fig.xaxis.axis_label = ""
                    fig.xaxis.visible = False
            else:
                fig.outline_line_color = None
            
            scatter_plots.append(fig)
    
    # xr = scatter_plots[0].x_range
    # yr = scatter_plots[0].y_range
    # for p in scatter_plots:
    #     p.x_range = xr
    #     p.y_range = yr
    
    grid = gridplot(scatter_plots, ncols = len(dataset.columns))
    show(grid)
    # save(grid)


In [ ]:
ppm = 1e6
y,x = 0,1

** Kepler Data Load **

In [ ]:
client = kplr.API()
koi3 = client.koi(3.01)

In [ ]:
print(koi3.koi_period, koi3.koi_period_err1)

In [ ]:
lcs3short = koi3.get_light_curves(short_cadence=True)

** Allocate All Data for Kepler-3 **

**Huber etal 2017**

In [ ]:
keplaunch = 2454833.0

# From Hubert et al. 2017
kep3period = 4.88782433
kep3t0     = 2454957.812464 - keplaunch
kep3aoR    = 14.64
kep3RpRs   = 0.05856
kep3FpFs   = 500/ppm
kep3inc    = 88.99
kep3ecc    = 0.26493
kep3omeg   = -162.149
kep3u1     = 0.646
kep3u2     = 0.048

In [ ]:
times, fluxs, ferrs = [], [], []
kep3_df_list = []
for lc in lcs3short:
    with lc.open() as f:
        if f[0].header['OBSMODE'] == 'short cadence':
            data     = f[1].data
            
            keepNow  = data["SAP_QUALITY"] == 0
            keepNow &= np.isfinite(data["TIME"])
            keepNow &= np.isfinite(data["PDCSAP_FLUX"])
            
            timesNow = np.ascontiguousarray(data["TIME"][keepNow], dtype=np.float64)
            fluxNow  = np.ascontiguousarray(data["PDCSAP_FLUX"][keepNow], dtype=np.float64)
            ferrNow  = np.ascontiguousarray(data["PDCSAP_FLUX_ERR"][keepNow], dtype=np.float64)
            
            phaseNow = ((timesNow - kep3t0) % kep3period)/kep3period
            phaseNow[phaseNow > .5] -= 1
            
            kep3_df           = DataFrame()
            kep3_df['time']   = timesNow
            kep3_df['flux']   = fluxNow
            kep3_df['ferr']   = ferrNow
            kep3_df['phase']  = phaseNow
            
            kep3_df_list.append(kep3_df)
            times.append(timesNow)
            fluxs.append(fluxNow)
            ferrs.append(ferrNow)

In [ ]:
jdintcpt = 2450000.0
kepStartEpoch = float(np.copy(kep3t0)) + jdintcpt

while kepStartEpoch > keplaunch:
    kepStartEpoch -= kep3period # start from way before kepler launched

kepStartEpoch -= jdintcpt
# iTran = 200
sliceWidth = 0.5

kep3_slice_df_list = []

# %matplotlib inline
# plt.figure(figsize=(10,10))
timeSlices, fluxSlices, ferrSlices, kep3Epochs = [],[],[],[]
for iEpoch in range(len(kep3_df_list)):#[2,9,29,33]:#
    # phases = (times[iEpoch] - kep3t0) % kep3period / kep3period
    nEpochsNow = (np.diff(kep3_df_list[iEpoch]['phase']) < -0.9).sum()
    if nEpochsNow:
        while kepStartEpoch < kep3_df_list[iEpoch]['time'].min():
            kepStartEpoch += kep3period
        
        for iTran in range(nEpochsNow):
            kep3epochKt0 = kepStartEpoch + iTran * kep3period
            transitSliceK = (kep3_df_list[iEpoch]['time'] > kep3epochKt0 - sliceWidth) & \
            (times[iEpoch] < kep3epochKt0 + sliceWidth)
            if np.sum(transitSliceK):
                timeSliceK = times[iEpoch][transitSliceK]
                timeSlices.append(np.linspace(np.nanmin(timeSliceK), np.nanmax(timeSliceK), timeSliceK.size))
                fluxSlices.append(fluxs[iEpoch][transitSliceK])
                ferrSlices.append(ferrs[iEpoch][transitSliceK])
                kep3Epochs.append(kep3epochKt0)
            else:
                print(iEpoch, iTran)
               # plt.errorbar(times[iEpoch], fluxs[iEpoch], ferrs[iEpoch]

In [ ]:
color_cycle

In [ ]:
# plt.figure(figsize=(9, 9))
i = 23
errorbars = bokeh_errorbars(kep3_df_list[i]['time'].values, 
                            kep3_df_list[i]['flux'].values, 
                            yerrs=kep3_df_list[i]['ferr'].values)

errorbars.xaxis.axis_label = "Time (KJD)"
errorbars.yaxis.axis_label = "Flux (Photons)"

errorbars.xaxis.axis_label_text_font = '20'
errorbars.yaxis.axis_label_text_font = '20'

show(errorbars); del errorbars

In [ ]:
plot_exist = False
tmin, tmax  = 291,322
for i in range(len(times)):
    timesi = kep3_df_list[i]['time']
    fluxsi = kep3_df_list[i]['flux']
    ferrsi = kep3_df_list[i]['ferr']
    useTime = (kep3_df_list[i]['time'] > tmin)&(kep3_df_list[i]['time'] < tmax)
    if sum(useTime):
        plot_exist= True
        errorbars = bokeh_errorbars(kep3_df_list[i]['time'][useTime], \
                                     kep3_df_list[i]['flux'][useTime], \
                                     kep3_df_list[i]['ferr'][useTime])

if plot_exist:
    errorbars.xaxis.axis_label = "Time (KJD)"
    errorbars.yaxis.axis_label = "Flux (Photons)"

    errorbars.xaxis.axis_label_text_font = '20'
    errorbars.yaxis.axis_label_text_font = '20'

    show(errorbars); del errorbars

** Old Planet Data Load **

** Phasing and Spliting Transits **

In [ ]:
i = 23
errorbars = bokeh_errorbars(kep3_df_list[i]['phase'], \
                            kep3_df_list[i]['flux'] , \
                            kep3_df_list[i]['ferr'] )

errorbars.xaxis.axis_label = "Time (KJD)"
errorbars.yaxis.axis_label = "Flux (Photons)"

errorbars.xaxis.axis_label_text_font = '20'
errorbars.yaxis.axis_label_text_font = '20'

show(errorbars); del errorbars

In [ ]:
i = 23
errorbars = bokeh_errorbars(kep3_df_list[i]['time'], \
                            kep3_df_list[i]['flux'] , \
                            kep3_df_list[i]['ferr'] )
# hline = Span(location=0, dimension='width', line_color='green', line_width=3)

vlines = []
for k in range(200, 206):
    vlines.append(Span(location=kep3t0 + k*kep3period, dimension='height', line_color=color_cycle[1], line_width=3))

errorbars.renderers.extend(vlines)

errorbars.xaxis.axis_label = "Time (KJD)"
errorbars.yaxis.axis_label = "Flux (Photons)"

errorbars.xaxis.axis_label_text_font = '20'
errorbars.yaxis.axis_label_text_font = '20'

show(errorbars); del errorbars

**Slice One Segment Transit**

In [ ]:
kTran = 201
kep3epochKt0 = kep3t0 + kTran * kep3period

In [ ]:
i = 23
sliceWidth = 0.5
transitSlice = (kep3_df_list[i]['time'].values > kep3epochKt0 - sliceWidth) * \
               (kep3_df_list[i]['time'].values < kep3epochKt0 + sliceWidth)

timeSliceK = kep3_df_list[i]['time'].values[transitSlice]
fluxSliceK = kep3_df_list[i]['flux'].values[transitSlice]
ferrSliceK = kep3_df_list[i]['ferr'].values[transitSlice]

timeSliceKmod = np.linspace(timeSliceK.min(), timeSliceK.max(), timeSliceK.size)

timeSliceKmod.size

In [ ]:
# plt.figure(figsize=(9,9))
errorbars = bokeh_errorbars(timeSliceK, fluxSliceK, ferrSliceK)
show(errorbars); del errorbars

**Outliers**

In [ ]:
fluxdiff = np.diff(fluxSliceK)

In [ ]:
# plt.figure(figsize = (9,9))
nsigma = 3
hlines = []
plot = figure()

hlines.append(Span(location=np.nanmedian(fluxdiff) + nsigma * np.nanstd(fluxdiff), 
                   dimension='width', line_color='black', line_width=3))
hlines.append(Span(location=np.nanmedian(fluxdiff) - nsigma * np.nanstd(fluxdiff), 
                   dimension='width', line_color='black', line_width=3))

hlines.append(Span(location=0, dimension='width', line_color='black', line_width=3))

plot.renderers.extend(hlines)

plot.line(x=np.arange(fluxdiff.size), y=fluxdiff)
show(plot); del plot

In [ ]:
itran=100
nsigma=3
outliers = np.where(abs(fluxdiff - np.nanmedian(fluxdiff)) > nsigma * np.nanstd(fluxdiff))[0]
print('Initial Outliers:', outliers)

for o in outliers:
    beforeOutlier = list(fluxSlices[itran][o - 10+1: o + 1])
    afterOutlier  = list(fluxSlices[itran][o + 1+1: o + 11+1])
    fluxSliceK[o] = np.median(beforeOutlier + afterOutlier)

fluxdiff = np.diff(fluxSliceK)
outliers = np.where(abs(fluxdiff - np.nanmedian(fluxdiff)) > nsigma * np.nanstd(fluxdiff))[0]
print('Final Outliers:', outliers)

In [ ]:
# plt.figure(figsize=(9,9));
errorbars = bokeh_errorbars(timeSliceK, fluxSliceK, ferrSliceK)
for o in outliers[1::2]:
    errorbars = bokeh_errorbars(timeSliceK[o], fluxSliceK[o], ferrSliceK[o], fig=errorbars, color='orange')

show(errorbars); del errorbars

In [ ]:
histogram = bokeh_hist(ferrSliceK[np.isfinite(ferrSliceK)],bins=ferrSliceK.size//10)
histogram.renderers.extend([Span(location=np.nanmedian(ferrSliceK), dimension='height', line_color='red', line_width=3)])
show(histogram);del histogram

In [ ]:
nanIndices  = np.where(np.isnan(fluxSliceK))[0]
medianDiff  = np.nanmedian(np.diff(fluxSliceK))

print('Initial NaNs:', nanIndices)

for n in nanIndices:
    beforeOutlier = list(fluxSlices[kTran][n - 10+1: n + 1])
    afterOutlier  = list(fluxSlices[kTran][n + 1+1 : n + 11+1])
    fluxSliceK[n] = median(beforeOutlier + afterOutlier)
    # fluxSliceK[n] = np.nanmedian(fluxSliceK)

print('Final NaNs:', np.where(np.isnan(fluxSliceK))[0])

In [ ]:
histogram = bokeh_hist(fluxSliceK, bins=fluxSliceK.size//10, range=[2.702e6, 2.707e6])
histogram.renderers.extend([Span(location=np.mean(fluxSliceK)  , dimension='height', line_color='red', line_width=3)])
histogram.renderers.extend([Span(location=np.median(fluxSliceK), dimension='height', line_color='orange', line_width=3)])
show(histogram);del histogram

In [ ]:
histogram = bokeh_hist(np.diff(fluxSliceK), bins=(fluxSliceK.size-1)//10, range=[-2000, 2000])
histogram.renderers.extend([Span(location=np.mean(np.diff(fluxSliceK)), dimension='height', line_color='red', line_width=3)])
histogram.renderers.extend([Span(location=np.median(np.diff(fluxSliceK)), dimension='height', line_color='orange', line_width=3)])
show(histogram);del histogram

In [ ]:
errorbars = bokeh_errorbars(timeSliceK, fluxSliceK, ferrSliceK)
for o in nanIndices:
    errorbars = bokeh_errorbars(timeSliceK[o], fluxSliceK[o], ferrSliceK[o],  color='orange', fig=errorbars)

show(errorbars);del errorbars

** Fitting the Transit with BATMAN **

In [ ]:
tparams     = batman.TransitParams()
tparams.t0  = kep3t0                     #time of inferior conjunction
tparams.per = kep3period                #orbital period 
tparams.rp  = kep3RpRs                   #planet radius (in units of stellar radii)
tparams.a   = kep3aoR                     #semi-major axis (in units of stellar radii)
tparams.inc = kep3inc                   #orbital inclination (in degrees)
tparams.ecc = kep3ecc                   #eccentricity
tparams.w   = kep3omeg                    #longitude of periastron (in degrees)
tparams.limb_dark = "quadratic"         #limb darkening model
tparams.u   = [kep3u1, kep3u2]            #limb darkening coefficients [u1, u2, u3, u4]

SliceKlcModel = batman.TransitModel(tparams, timeSliceKmod, transittype='primary')

In [ ]:
lckep3Model = SliceKlcModel.light_curve(tparams)

In [ ]:
errorbars = bokeh_errorbars(timeSliceK, fluxSliceK / np.median(fluxSliceK), ferrSliceK / np.median(fluxSliceK))
errorbars.line(timeSliceKmod, lckep3Model, color='orange')

errorbars.xaxis.axis_label = "Time From Central Transit (KJD)"
errorbars.yaxis.axis_label = "Relative Flux (Photons)"
errorbars.xaxis.axis_label_text_font_size = '20px'
errorbars.yaxis.axis_label_text_font_size = '20px'

show(errorbars);del errorbars

In [ ]:
initialParams = Parameters()

initialParams.add_many(
    ('period'   , kep3period, False),
    ('tCenter'  , kep3t0    , True , kep3t0 - 0.1, kep3t0 + 0.1),
    ('inc'      , kep3inc   , False, 0.0, 90. ),
    ('aprs'     , kep3aoR   , False, 0.0, 100.),
    ('tdepth'   , kep3RpRs  , True , 0.0, 1.0 ),
    ('edepth'   , kep3FpFs  , True , 0.0, 1.0 ),
    ('ecc'      , kep3ecc   , False, 0.0, 1.0 ),
    ('omega'    , kep3omeg  , False, 0.0, 1.0 ),
    ('u1'       , kep3u1    , True , 0.0, 1.0 ),
    ('u2'       , kep3u2    , True , 0.0, 1.0 ),
    ('intcpt'   , 1.0       , True ),
    ('slope'    , 0.0       , True ),
    ('crvtur', 0.0          , True ))

In [ ]:
partial_residuals = partial(residuals_func, 
                            times  = timeSliceKmod, 
                            flux   = fluxSliceK / np.median(fluxSliceK), 
                            fluxerr= ferrSliceK / np.median(fluxSliceK)
                            )

mle0  = Minimizer(partial_residuals, initialParams)

start = time()
fitResult = mle0.leastsq()
print("LMFIT operation took {} seconds".format(time()-start))

In [ ]:
report_errors(fitResult.params)

In [ ]:
errorbars = bokeh_errorbars(timeSliceKmod, fluxSliceK / np.median(fluxSliceK), ferrSliceK / np.median(fluxSliceK))
errorbars.line(x=timeSliceKmod, y=transit_line_model(fitResult.params, timeSliceKmod), color='orange', line_width=3)

errorbars.xaxis.axis_label = "Time From Central Transit (KJD)"
errorbars.yaxis.axis_label = "Relative Flux (Photons)"
errorbars.xaxis.axis_label_text_font_size = '20px'
errorbars.yaxis.axis_label_text_font_size = '20px'

show(errorbars);del errorbars

In [ ]:
errorbars = bokeh_errorbars(timeSliceKmod, 
                            fluxSliceK / np.median(fluxSliceK) - transit_line_model(fitResult.params, timeSliceKmod), 
                            ferrSliceK / np.median(fluxSliceK))
errorbars.line(x=timeSliceKmod, y=np.zeros(timeSliceKmod.size), color='orange', line_width=3)

errorbars.xaxis.axis_label = "Time From Central Transit (KJD)"
errorbars.yaxis.axis_label = "Relative Flux (Photons)"
errorbars.xaxis.axis_label_text_font_size = '20px'
errorbars.yaxis.axis_label_text_font_size = '20px'

show(errorbars);del errorbars

In [ ]:
residuals = fluxSliceK - transit_line_model(fitResult.params, timeSliceKmod)
chisq     = np.sum((residuals / ferrSliceK)**2.)
print(chisq / residuals.size)

In [ ]:
def lnprob(p):
    resid = partial_residuals(p)
    s = p['f']
    resid *= 1 / s
    resid *= resid
    resid += np.log(2 * np.pi * s**2)
    return -0.5 * np.sum(resid)

In [ ]:
mle0.params.add('f', value=1, min=0.001, max=2)

In [ ]:
mini  = Minimizer(lnprob, mle0.params)

In [ ]:
start = time()

res   = mini.emcee(params=mle0.params, steps=100, nwalkers=100, burn=1, thin=10, ntemps=1, 
                    pos=None, reuse_sampler=False, workers=1, float_behavior='posterior', 
                    is_weighted=True, seed=None)

joblib.dump(res, 'emcee_results.joblib.save')
print("MCMC operation took {} seconds".format(time()-start))

In [ ]:
initialParams = Parameters()

initialParams.add_many(
    ('period'   , kep3period, False),
    ('tCenter'  , kep3t0    , True , kep3t0 - 0.1, kep3t0 + 0.1),
    ('inc'      , kep3inc   , False, 0.0, 90. ),
    ('aprs'     , kep3aoR   , False, 0.0, 100.),
    ('tdepth'   , kep3RpRs  , True , 0.0, 1.0 ),
    ('edepth'   , kep3FpFs  , True , 0.0, 1.0 ),
    ('ecc'      , kep3ecc   , False, 0.0, 1.0 ),
    ('omega'    , kep3omeg  , False, 0.0, 1.0 ),
    ('u1'       , kep3u1    , True , 0.0, 1.0 ),
    ('u2'       , kep3u2    , True , 0.0, 1.0 ),
    ('intcpt'   , 1.0       , True ),
    ('slope'    , 0.0       , True ),
    ('crvtur', 0.0          , True ))

In [ ]:
res_flatchain.shape

In [ ]:
# corner_use    = [1, 4,5,]
res_var_names = np.array(res.var_names)
res_flatchain = np.array(res.flatchain)
res_df = DataFrame(res_flatchain, columns=res_var_names)
# res_flatchain.T[corner_use].shape

In [ ]:
bokeh_corner_plot(res_df)